In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
from prettytable import PrettyTable

# Connect to db

In [2]:
%load_ext dotenv
%dotenv

In [3]:
connection = psycopg2.connect(
        host=os.getenv('DATABASE_HOST'),
        port=os.getenv('DATABASE_PORT'),
        user=os.getenv('DATABASE_USER'),
        password=os.getenv('DATABASE_PASSWORD'),
        dbname=os.getenv('DATABASE_NAME'),
    )
connection.autocommit = True
cursor = connection.cursor()

In [4]:
def select_query(query):
    cursor.execute(query)
    
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]

    # format the output nicely
    table = PrettyTable()
    table.field_names = columns
    for row in rows:
        table.add_row(row)
                      
    return table

In [5]:
def other_queries(query):
    cursor.execute(query)

# Invistigate db

![](images/Star.png)

In [8]:
query = ("""
    select * from dimDate limit 3;
""")
select_query(query)

date_key,date,year,quarter,month,week,day,is_weekend
20070320,2007-03-20,2007,1,3,12,20,False
20070129,2007-01-29,2007,1,1,5,29,False
20070406,2007-04-06,2007,2,4,14,6,False


In [10]:
query = ("""
    select * from dimCustomer limit 3;
""")
select_query(query)

customer_key,customer_id,first_name,last_name,email,address,address2,district,city,country,postal_code,phone,active,create_date,start_date,end_date
1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1913 Hanoi Way,,Nagasaki,Sasebo,Japan,35200,28303384290,1,2006-02-14 00:00:00,2024-12-04,2024-12-04
2,2,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,1121 Loja Avenue,,California,San Bernardino,United States,17886,838635286649,1,2006-02-14 00:00:00,2024-12-04,2024-12-04
3,3,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,692 Joliet Street,,Attika,Athenai,Greece,83579,448477190408,1,2006-02-14 00:00:00,2024-12-04,2024-12-04


In [11]:
query = ("""
    select * from dimMovie limit 3;
""")
select_query(query)

movie_key,film_id,title,description,release_year,language,original_language,rental_duration,length,rating,special_features
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,English,English,6,86,PG,"{""Deleted Scenes"",""Behind the Scenes""}"
2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,English,English,3,48,G,"{Trailers,""Deleted Scenes""}"
3,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,English,English,7,50,NC-17,"{Trailers,""Deleted Scenes""}"


In [12]:
query = ("""
    select * from dimStore limit 3;
""")
select_query(query)

store_key,store_id,address,address2,district,city,country,postal_code,manager_first_name,manager_last_name,start_date,end_date
1,1,47 MySakila Drive,None,Alberta,Lethbridge,Canada,,Mike,Hillyer,2024-12-04,2024-12-04
2,2,28 MySQL Boulevard,None,QLD,Woodridge,Australia,,Jon,Stephens,2024-12-04,2024-12-04


In [13]:
query = ("""
    select * from factSales limit 3;
""")
select_query(query)

sales_key,date_key,customer_key,movie_key,store_key,sales_amount
1,20070124,269,870,2,1.99
2,20070125,269,651,1,0.99
3,20070128,269,818,1,6.99


In [16]:
query = ("""
    SELECT
        dimdate.day AS day,
        dimmovie.rating AS rating,
        dimcustomer.city AS city,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate     ON (dimdate.date_key = factsales.date_key)
        JOIN dimcustomer ON (dimcustomer.customer_key = factsales.customer_key)
        JOIN dimmovie    ON (dimmovie.movie_key = factsales.movie_key)
    GROUP BY
        (dimdate.day, dimmovie.rating, dimcustomer.city)
    ORDER BY
        revenue DESC
    LIMIT 20
""")
select_query(query)

day,rating,city,revenue
30,G,San Bernardino,24.97
30,NC-17,Apeldoorn,23.95
21,NC-17,Belm,22.97
28,R,Mwanza,21.97
30,PG-13,Zanzibar,21.97
21,G,Citt del Vaticano,21.97
1,R,Qomsheh,19.97
17,G,Rajkot,19.97
28,PG-13,Dhaka,19.97
22,R,Yangor,19.97


# Slicing
- Slicing is the reduction of the dimensionality of a cube by 1 e.g. 3 dimensions to 2, fixing one of the dimensions to a single value. In the example above, we have a 3-dimensional cube on day, rating, and city.
- Below `rating` is fixced to `PG-13` which reduces the dimensionality
- The acutal relation we have now is between the `day` and the `city` given that the rating is `PG-13` constant

In [17]:
query = ("""
    SELECT
        dimdate.day AS day,
        dimmovie.rating AS rating,
        dimcustomer.city AS city,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate     ON (dimdate.date_key = factsales.date_key)
        JOIN dimcustomer ON (dimcustomer.customer_key = factsales.customer_key)
        JOIN dimmovie    ON (dimmovie.movie_key = factsales.movie_key)
    WHERE
        dimmovie.rating = 'PG-13'
    GROUP BY
        (dimdate.day, dimmovie.rating, dimcustomer.city)
    ORDER BY
        revenue DESC
    LIMIT 20
""")
select_query(query)

day,rating,city,revenue
30,PG-13,Zanzibar,21.97
28,PG-13,Dhaka,19.97
30,PG-13,Osmaniye,18.97
29,PG-13,Shimoga,18.97
21,PG-13,Asuncin,18.95
21,PG-13,Parbhani,17.98
20,PG-13,Baha Blanca,17.98
30,PG-13,Nagareyama,17.98
30,PG-13,Tanauan,17.96
17,PG-13,Ikerre,17.95


# Dicing
Dicing is creating a subcube with the same dimensionality but fewer values for two or more dimensions.
    
- `ratings` of PG or PG-13
- `city` of Bellevue or Lancaster
- `day` equal to 1, 15, or 30

In [18]:
query = ("""
    SELECT
        dimdate.day AS day,
        dimmovie.rating AS rating,
        dimcustomer.city AS city,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate     ON (dimdate.date_key = factsales.date_key)
        JOIN dimcustomer ON (dimcustomer.customer_key = factsales.customer_key)
        JOIN dimmovie    ON (dimmovie.movie_key = factsales.movie_key)
    WHERE
        dimmovie.rating  in ('PG', 'PG-13')
    AND
        dimcustomer.city in ('Bellevue', 'Lancaster')
    AND
        dimdate.day      in ('1', '15', '30')
    GROUP BY
        (dimdate.day, dimmovie.rating, dimcustomer.city)
    ORDER BY
        revenue DESC
    LIMIT 20
""")
select_query(query)

day,rating,city,revenue
30,PG,Lancaster,12.98
1,PG-13,Lancaster,5.99
30,PG-13,Bellevue,3.99
30,PG-13,Lancaster,2.99
15,PG-13,Bellevue,1.98
1,PG,Bellevue,0.99


# Roll-up
- Stepping up the level of aggregation to a large grouping
- `city` -> `country`

In [19]:
query = ("""
    SELECT
        dimdate.day AS day,
        dimmovie.rating AS rating,
        dimcustomer.country AS country,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate     ON (dimdate.date_key = factsales.date_key)
        JOIN dimcustomer ON (dimcustomer.customer_key = factsales.customer_key)
        JOIN dimmovie    ON (dimmovie.movie_key = factsales.movie_key)
    GROUP BY
        (dimdate.day, dimmovie.rating, dimcustomer.country)
    ORDER BY
        revenue DESC
    LIMIT 20
""")
select_query(query)

day,rating,country,revenue
30,G,China,169.67
30,PG,India,156.67
30,NC-17,India,153.64
30,PG-13,China,146.67
30,R,China,145.66
30,R,India,143.68
30,G,India,137.67
18,NC-17,India,135.75
30,PG,China,131.72
21,PG-13,India,128.74


# Drill-down

- Breaking up one of the dimensions to a lower level.
- `city` -> `districts`

In [20]:
query = ("""
    SELECT
        dimdate.day AS day,
        dimmovie.rating AS rating,
        dimcustomer.district AS district,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate     ON (dimdate.date_key = factsales.date_key)
        JOIN dimcustomer ON (dimcustomer.customer_key = factsales.customer_key)
        JOIN dimmovie    ON (dimmovie.movie_key = factsales.movie_key)
    GROUP BY
        (dimdate.day, dimmovie.rating, dimcustomer.district)
    ORDER BY
        revenue DESC
    LIMIT 20
""")
select_query(query)

day,rating,district,revenue
30,PG-13,Southern Tagalog,53.88
30,G,Inner Mongolia,38.93
30,G,Shandong,36.93
30,NC-17,West Bengali,36.92
17,PG-13,Shandong,34.95
1,PG,California,32.94
18,NC-17,So Paulo,32.93
21,R,So Paulo,31.93
30,NC-17,Buenos Aires,31.93
30,PG,Southern Tagalog,30.94


# Grouping Sets
It happens often that for 3 dimensions, you want to aggregate a fact:
- by nothing (total)
- then by the 1st dimension
- then by the 2nd
- then by the 3rd
- then by the 1st and 2nd
- then by the 2nd and 3rd
- then by the 1st and 3rd
- then by the 1st and 2nd and 3rd

Since this is very common, and in all cases, we are iterating through all the fact table anyhow, there is a more clever way to do that using the SQL grouping statement `GROUPING SETS`



## Total Revenue

In [24]:
query = ("""
    SELECT SUM(sales_amount) as totla_revenue from factsales
""")
select_query(query)

totla_revenue
67416.51


## Revenue by Country

In [25]:
query = ("""
    SELECT
        dimstore.country AS country,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimstore ON (dimstore.store_key = factsales.store_key)
    GROUP BY
        dimstore.country
""")
select_query(query)

country,revenue
Canada,33689.74
Australia,33726.77


## Revenue by Month

In [26]:
query = ("""
    SELECT
        dimdate.month AS month,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate ON (dimdate.date_key = factsales.date_key)
    GROUP BY
        dimdate.month
""")
select_query(query)

month,revenue
3,23886.56
5,514.18
4,28559.46
2,9631.88
1,4824.43


# Revenue by Month & Country

In [27]:
query = ("""
    SELECT
        dimdate.month AS month,
        dimstore.country AS country,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate ON (dimdate.date_key = factsales.date_key)
        JOIN dimstore ON (dimstore.store_key = factsales.store_key)
    GROUP BY
        dimdate.month,
        dimstore.country
    ORDER BY
        dimdate.month,
        dimstore.country,
        revenue DESC
""")
select_query(query)

month,country,revenue
1,Australia,2364.19
1,Canada,2460.24
2,Australia,4895.10
2,Canada,4736.78
3,Australia,12060.33
3,Canada,11826.23
4,Australia,14136.07
4,Canada,14423.39
5,Australia,271.08
5,Canada,243.10


## Revenue Total, by Month, by Country, by Month & Country All in one shot 
`GROUP BY grouping sets`
- The `None` here do mean something, they are a comulative sum

In [30]:
query = ("""
    SELECT
        dimdate.month AS month,
        dimstore.country AS country,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate ON (dimdate.date_key = factsales.date_key)
        JOIN dimstore ON (dimstore.store_key = factsales.store_key)
    GROUP BY
        grouping sets ((), dimdate.month, dimstore.country, (dimdate.month, dimstore.country))
""")
select_query(query)

month,country,revenue
None,None,67416.51
5,Australia,271.08
4,Australia,14136.07
1,Canada,2460.24
1,Australia,2364.19
4,Canada,14423.39
2,Canada,4736.78
2,Australia,4895.10
3,Australia,12060.33
3,Canada,11826.23


# CUBE
- Group by `CUBE (dim1, dim2, ..)` , produces all combinations of different lenghts in one go.
- This view could be materialized in a view and queried which would save lots repetitive aggregations
- Equivalente to above, simpler and faster
- The same applies with `NULLs` here too, but be careful before running `cube` make sure that the original data doesn't have any null, usally one clean and drop any nulls before running `cube`

In [29]:
query = ("""
    SELECT
        dimdate.month AS month,
        dimstore.country AS country,
        SUM(factsales.sales_amount) AS revenue
    FROM
        factsales 
        JOIN dimdate ON (dimdate.date_key = factsales.date_key)
        JOIN dimstore ON (dimstore.store_key = factsales.store_key)
    GROUP BY
        cube(dimdate.month, dimstore.country)
""")
select_query(query)

month,country,revenue
None,None,67416.51
5,Australia,271.08
4,Australia,14136.07
1,Canada,2460.24
1,Australia,2364.19
4,Canada,14423.39
2,Canada,4736.78
2,Australia,4895.10
3,Australia,12060.33
3,Canada,11826.23
